<a href="https://colab.research.google.com/github/quiquefraga/obama_agent/blob/main/ObamaTweetAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!pip install openai


In [78]:
!pip install feedparser


In [79]:
!pip install tiktoken

In [80]:
!pip install -U langchain-community

In [81]:
!pip install pandas chromadb langchain openai


In [82]:
import pandas as pd
import chromadb
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import feedparser

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import gradio as gr


In [83]:
import os
from google.colab import userdata

# Recuperar la API Key desde Colab Secrets
api_key = userdata.get('quiqueapi')

# Asignarla a la variable de entorno que OpenAI necesita
os.environ["OPENAI_API_KEY"] = api_key

# Verificar que se guardó correctamente
print("✅ API Key configurada:", os.getenv("OPENAI_API_KEY") is not None)


✅ API Key configurada: True


In [84]:
# Ruta del archivo CSV en Colab
archivo_csv = "/content/tweets.csv"  # Ajustá esto si el nombre es distinto



In [85]:
# Cargar el CSV en un DataFrame
df = pd.read_csv(archivo_csv)



In [86]:
# Verificar las primeras filas
print(df.head())



             tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  798914779213500422                    NaN                  NaN   
1  798909747357450240                    NaN                  NaN   
2  798908490152448001                    NaN                  NaN   
3  798907196469870592                    NaN                  NaN   
4  798899681472421888                    NaN                  NaN   

                   timestamp  \
0  2016-11-16 15:45:01 +0000   
1  2016-11-16 15:25:01 +0000   
2  2016-11-16 15:20:01 +0000   
3  2016-11-16 15:14:53 +0000   
4  2016-11-16 14:45:01 +0000   

                                              source  \
0  <a href="https://about.twitter.com/products/tw...   
1  <a href="https://about.twitter.com/products/tw...   
2  <a href="https://about.twitter.com/products/tw...   
3  <a href="https://about.twitter.com/products/tw...   
4  <a href="https://about.twitter.com/products/tw...   

                                                text  r

In [87]:
# Suponiendo que la columna con los tuits se llama "tweet"
tuits = df["text"].dropna().tolist()  # Eliminar valores nulos por seguridad



In [88]:
# Inicializar ChromaDB en memoria (para Colab no necesitamos "PersistentClient")
chroma_client = chromadb.PersistentClient(path="./chroma_db")



In [89]:
# Crear embeddings con OpenAI (necesitás una API Key de OpenAI)
embedding_function = OpenAIEmbeddings(
    openai_api_key=api_key,  # Pass the api_key directly here
    model="text-embedding-ada-002"
)

In [90]:

# Indexar los tuits en ChromaDB
vectorstore = Chroma.from_texts(tuits, embedding_function, collection_name="obama_tweets", client=chroma_client)



In [54]:
# Buscar tuits relevantes para un input
#consulta = "Climate change policy in the United States"  # Un ejemplo de noticia

# Recuperar los 3 tuits más relevantes
#resultados = vectorstore.similarity_search(consulta, k=3)

#print("🔍 Tuits más relevantes sobre el tema:")
#for i, doc in enumerate(resultados):
 #   print(f"{i+1}. {doc.page_content}")


In [91]:
# URL del feed RSS del NYT (se puede cambiar por otra categoría)
rss_url = "https://rss.nytimes.com/services/xml/rss/nyt/World.xml"


In [92]:

# Obtener el feed
feed = feedparser.parse(rss_url)


In [93]:

# Extraer los primeros 5 titulares y resúmenes
noticias = []
for entry in feed.entries[:5]:  # Tomamos solo las 5 más recientes
    titulo = entry.title
    resumen = entry.summary
    noticias.append({"titulo": titulo, "resumen": resumen})


In [94]:

# Mostrar noticias
for i, noticia in enumerate(noticias):
    print(f"📰 Noticia {i+1}: {noticia['titulo']}")  # Closing bracket added after 'titulo'
    print(f"   Resumen: {noticia['resumen']}")  # Print the summary as well

📰 Noticia 1: Ukraine Targets Moscow With Large-Scale Drone Attack
   Resumen: The assault, which the mayor called the largest on Russia’s capital since the war began, was a reminder of Ukraine’s power to strike as its president proposes an air truce.
📰 Noticia 2: Ukraine and U.S. Talks on War Begin as Gulfs Remain
   Resumen: American and Ukrainian representatives are meeting in Saudi Arabia, but the United States, Ukraine and Russia envisage very different paths to peace.
📰 Noticia 3: Ramadan in the West Bank: Displacement and Despair
   Resumen: An Israeli military operation has uprooted tens of thousands of Palestinians who can’t break their fast in their own homes and don’t know when, or if, they will ever return.
📰 Noticia 4: Philippines’ Ex-President Duterte Arrested Under I.C.C. Warrant
   Resumen: Rodrigo Duterte, the former Philippine president, was arrested on Tuesday in Manila and was flown to The Hague to face International Criminal Court charges of crimes against humanity.

In [95]:
# Buscar tuits relevantes para cada noticia y mostrar resultados
for noticia in noticias:
    consulta = noticia["titulo"]  # Usamos el título de la noticia como consulta
    resultados = vectorstore.similarity_search(consulta, k=3)  # Buscar los 3 tuits más similares

    print(f"🔍 Opinión sobre: {noticia['titulo']}")
    print(f"📝 Resumen: {noticia['resumen']}\n")

    print("💬 Tuits relevantes:")
    for i, doc in enumerate(resultados):
        print(f"{i+1}. {doc.page_content}")

    print("-" * 50)


🔍 Opinión sobre: Ukraine Targets Moscow With Large-Scale Drone Attack
📝 Resumen: The assault, which the mayor called the largest on Russia’s capital since the war began, was a reminder of Ukraine’s power to strike as its president proposes an air truce.

💬 Tuits relevantes:
1. "The world is watching reports of a downed passenger jet near the Russia-Ukraine border." —President Obama
2. "The world is watching reports of a downed passenger jet near the Russia-Ukraine border." —President Obama
3. "This ongoing Russian incursion into Ukraine will only bring more costs and consequences for Russia." —President Obama
--------------------------------------------------
🔍 Opinión sobre: Ukraine and U.S. Talks on War Begin as Gulfs Remain
📝 Resumen: American and Ukrainian representatives are meeting in Saudi Arabia, but the United States, Ukraine and Russia envisage very different paths to peace.

💬 Tuits relevantes:
1. "We agree on the need to maintain strong sanctions against Russia until it end

In [107]:
# Configurar GPT-4o Mini
chat_model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

# Generar opinión basada en los tuits y la noticia
for noticia in noticias:
    consulta = noticia["titulo"]
    resumen = noticia["resumen"]

    # Buscar tuits relevantes en ChromaDB
    resultados = vectorstore.similarity_search(consulta, k=3)
    tuits_relevantes = "\n".join([doc.page_content for doc in resultados])

    # Construir el prompt
    prompt = f"""
    Basado en la siguiente noticia y en los tuits de Barack Obama, genera una opinión en su estilo:

    📰 Noticia: {consulta}
    📝 Resumen: {resumen}

    📢 Tuits relevantes:
    {tuits_relevantes}

    Responde como si fueras Barack Obama.
    """

    # Generar respuesta con GPT-4o Mini
    respuesta = chat_model.invoke([HumanMessage(content=prompt)])

    # Mostrar la opinión generada
    print(f"🗣️ Opinión generada para: {consulta}")
    print(respuesta.content)
    print("=" * 80)


🗣️ Opinión generada para: Ukraine Targets Moscow With Large-Scale Drone Attack
La reciente ofensiva de Ucrania sobre Moscú es un recordatorio poderoso de la resiliencia y la determinación del pueblo ucraniano. A medida que enfrentan una agresión injusta, su capacidad para responder demuestra que la lucha por la soberanía y la libertad no se apaga, incluso en los momentos más oscuros. 

Es vital que continuemos apoyando a Ucrania en su búsqueda de paz y justicia, y que respaldemos los esfuerzos de diálogo y negociación, como la propuesta de un alto el fuego. La comunidad internacional debe estar unida en su condena a la violencia y en la promoción de soluciones pacíficas. 

Los costos de esta guerra son significativos, no solo para Ucrania, sino también para Rusia y el orden mundial. No podemos permitir que la agresión se normalice. Es hora de que todos los líderes se comprometan a trabajar juntos por un futuro más pacífico y seguro para todos.
🗣️ Opinión generada para: Ukraine and U.S.

In [97]:
!pip install gradio


In [111]:
# Función para obtener noticias y generar opiniones
def generar_opiniones():
    # Obtener noticias del NYT
    rss_url = "https://rss.nytimes.com/services/xml/rss/nyt/World.xml"
    feed = feedparser.parse(rss_url)
    noticias = feed.entries[:5]  # Obtener 5 noticias más recientes

    resultados = []
    for noticia in noticias:
        consulta = noticia["title"]
        resumen = noticia["summary"]

        # Buscar tuits relevantes en ChromaDB
        resultados_tuits = vectorstore.similarity_search(consulta, k=3)
        tuits_relevantes = "\n".join([doc.page_content for doc in resultados_tuits])

        # Construir el prompt
        prompt = f"""
        Basado en la siguiente noticia y en los tuits de Barack Obama, genera una opinión en su estilo:

        📰 Noticia: {consulta}
        📝 Resumen: {resumen}

        📢 Tuits relevantes:
        {tuits_relevantes}

        Responde como si fueras Barack Obama.
        """

        # Generar respuesta con GPT-4o Mini
        respuesta = chat_model.invoke([HumanMessage(content=prompt)])

        resultados.append(f"🗞️ {consulta}\n📢 Opinión de Obama Tweet Agent:\n{respuesta.content}\n" + "="*80)

    return "\n\n".join(resultados)

# Crear la interfaz con Gradio
interface = gr.Interface(
    fn=generar_opiniones,  # Ejecuta la función cuando se presiona el botón
    inputs=[],
    outputs=gr.Textbox(label="Opiniones Generadas", lines=20),
    title="📰 Obama Tweet Agent",
    description="Este agente IA analiza noticias del New York Times y genera opiniones basadas en los tuits del presidente Barack Obama entre 2009-2017. Fuente tweets: https://www.obamalibrary.gov/"
)


In [112]:
# Ejecutar la app
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://42b8e0348e60d067e2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
